# Interactive Plotting with Bokeh
* Author: Johannes Maucher
* Last update: 03.05.2018

In [39]:
import pandas as pd
import numpy as np
#import bokeh
from IPython.display import display
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource,HoverTool

import json

In [40]:
output_notebook()

Loading BokehJS ...

In [41]:
from xml.etree.ElementTree import fromstring
from time import strptime, strftime
from optparse import OptionParser
import sys
import re

In [42]:
def findtext(e, name, default=None):
    """
    findtext
    helper function to find sub-element of e with given name and
    return text value
    returns default=None if sub-element isn't found
    """
    try:
        return e.find(name).text
    except:
        return default

In [43]:
def parsetcx(xml):
    """
    parsetcx
    parses tcx data, returning a list of all Trackpoints where each
    point is a tuple of 
      (activity, lap, timestamp, seconds, lat, long, alt, dist, heart, cad)
    xml is a string of tcx data
    """

    # remove xml namespace (xmlns="...") to simplify finds
    # note: do this using ET._namespace_map instead?
    # see http://effbot.org/zone/element-namespaces.htm
    xml = re.sub('xmlns=".*?"','',xml)

    # parse xml
    tcx=fromstring(xml)

    activity = tcx.find('.//Activity').attrib['Sport']
    sample=1
    lapnum=1
    points=[]
    for lap in tcx.findall('.//Lap/'):
        for point in lap.findall('.//Trackpoint'):
            # time, both as string and in seconds GMT
            # note: adjust for timezone?
            timestamp = findtext(point, 'Time')
            if timestamp:
                time = strftime('%X', strptime(timestamp[:-1],'%Y-%m-%dT%H:%M:%S.%f'))
                date = strftime('%x', strptime(timestamp[:-1],'%Y-%m-%dT%H:%M:%S.%f'))
                #print(strptime(timestamp[:-1],'%Y-%m-%dT%H:%M:%S.%f')[3])
            else:
                time = None
            # cummulative distance
            dist = findtext(point, 'DistanceMeters')
                
            # latitude and longitude
            position = point.find('Position')
            lat = findtext(position, 'LatitudeDegrees')
            long = findtext(position, 'LongitudeDegrees')

            # altitude
            alt = float(findtext(point, 'AltitudeMeters',0))
            
            # heart rate
            heart = int(findtext(point.find('HeartRateBpm'),'Value',0))

            # cadence
            cad = int(findtext(point, 'Cadence',0))

            # append to list of points
            points.append((sample,
                           activity,
                           lapnum,
                           date, 
                           time, 
                           lat,
                           long,
                           alt,
                           dist,
                           heart,
                           cad))
            sample+=1
        # next lap
        lapnum+=1
    return points

In [44]:
columnIndex={"sample":0,"date":3,"time":4,'lat':5,"long":6,"alt":7,"dist":8,"heart":9,"cad":10}

In [45]:
def createDataframe(pointslist,columnIndex):
    datadict={}
    for col in columnIndex.keys():
        datadict[col]=[]
    for row in pointslist:
        for col in columnIndex.keys(): 
            value=row[columnIndex[col]]
            if value == None:
                try:
                    value = datadict[col][-1]
                except:
                    value=0
            datadict[col].append(value)
    dataframe = pd.DataFrame(data=datadict,columns=columnIndex.keys(),index=datadict["sample"])
    #return dataframe.drop(['sample'],axis=1)
    return dataframe

In [46]:
istream = open('trainingdata.tcx','r')
# read xml contents and parse tcx
xml = istream.read()
points = parsetcx(xml)

In [47]:
trainingDF=createDataframe(points,columnIndex)

In [48]:
print(trainingDF.head())

   sample      date      time          lat        long  alt  \
1       1  04/14/18  15:25:57  48.74112833  9.10069667  0.0   
2       2  04/14/18  15:25:58  48.74112833  9.10069667  0.0   
3       3  04/14/18  15:25:59  48.74112667  9.10066667  0.0   
4       4  04/14/18  15:26:00  48.74112667  9.10063167  0.0   
5       5  04/14/18  15:26:01  48.74112333  9.10059167  0.0   

                 dist  heart  cad  
1                 0.5     88    0  
2                 0.5     89    0  
3                 0.5     89    0  
4  0.6000000238418579     91    0  
5   3.200000047683716     92    0  


In [55]:
HRZ=[93,112,130,149,167,186]
HRC=["grey","blue","green","yellow","red"]

In [59]:
from bokeh.layouts import gridplot
from bokeh.models import BoxAnnotation, Range1d

source = ColumnDataSource(trainingDF)

options = dict(plot_width=800, plot_height=300,
               tools="pan,wheel_zoom,box_zoom,box_select,lasso_select,reset")

p1 = figure(title="heartrate over time", **options)
p1.line("sample","heart", color="blue", source=source)
for i,col in enumerate(HRC):
    p1.add_layout(BoxAnnotation(bottom=HRZ[i],top=HRZ[i+1], fill_alpha=0.3, fill_color=col))

p2 = figure(title="cadence over time",x_range=p1.x_range,y_range=Range1d(60,90), **options)
p2.line("sample","cad", color="green", source=source)

p3 = figure(title="altitude over time",x_range=p1.x_range,y_range=Range1d(300,600), **options)
p3.line("sample","alt", color="red", source=source)


p = gridplot([[p1],[p2],[p3]], toolbar_location="right")

show(p)

In [60]:
with open('gmapsKey.json') as f:
    gmkey = json.load(f)

In [67]:
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

output_file("gmap.html")

map_options = GMapOptions(lat=float(trainingDF["lat"][1]), lng=float(trainingDF["long"][1]), map_type="roadmap", zoom=11)

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
p = gmap(gmkey['gmaps'], map_options, title="Stuttgart")

SS=20 # subsampling factor -> not all gps points must be drawn 
source = ColumnDataSource(
    data = dict(lat=trainingDF["lat"].values[::SS].tolist(),
               lon=trainingDF["long"].values[::SS].tolist())
)
p.circle(x="lon", y="lat", size=5, fill_color="red", fill_alpha=0.8, source=source)

show(p)